In [ ]:
!pip install huggingface_hub --upgrade
!pip install diffusers accelerate transformers safetensors sentencepiece
!pip install clean-fid

In [ ]:
import matplotlib.pyplot as plt
from diffusers import StableDiffusionPipeline
import torch
from huggingface_hub import login
import numpy as np
import os
from PIL import Image
import json
import os.path
from cleanfid import fid

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

We use s Stable Diffusion model from Hugging Face pretrained on the LAION 5B dataset.

In [ ]:
# Login to Hugging Face
login()

In [ ]:
#Initialize Stable Diffusion model
model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda")

We can generate 5 images using Stable Diffusion given prompts from our dataset and compare them with the true images.

In [ ]:
with open("/content/drive/MyDrive/Spring Quarter/CSE493G1/Final Project/data/prompts.json", "r") as file:
  prompts = json.load(file)

dir_name = "/content/drive/MyDrive/Spring Quarter/CSE493G1/Final Project/data/outputs/"
if not os.path.exists(dir_name):
  os.makedirs(dir_name)

images_path = '/content/drive/MyDrive/Spring Quarter/CSE493G1/Final Project/data/images/'

# List of sample true images
image_names = ['37i9dQZF1DWWOaP4H0w5b0_0.jpg', '43Nvl9B8fErDqqPSx1OdW0.jpg', '0Iy20da7GRpAQGfVXZ5CFB.jpg', '216GOIIed7qbRTVVDDDES4.jpg', '37i9dQZF1DWV7cvDzE3MOI.jpg']
images = []

# Save the true images
for i in range(len(image_names)):
  image = Image.open(images_path + image_names[i])
  images.append(image)
  plt.imshow(image)
  image.save(dir_name + image_names[i])

# Generate and save the output images as well as the prompts
for i in range(len(image_names)):
  image_id = image_names[i][0:22]
  prompt = prompts[image_id]
  print(prompt)
  generated_image = pipe(prompt).images[0]
  plt.imshow(generated_image)
  generated_image.save(dir_name + image_names[i][:-4] + "generated.jpg")

We can run Stable Diffusion with the prompts from our test set to determine how well the model performs based off of the FID score with the true images. 

In [ ]:
dir_orig = "/content/drive/MyDrive/Spring Quarter/CSE493G1/Final Project/data/Stable Diffusion Baseline/Original/"
if not os.path.exists(dir_orig):
  os.makedirs(dir_orig)

dir_gen = "/content/drive/MyDrive/Spring Quarter/CSE493G1/Final Project/data/Stable Diffusion Baseline/Generated/"
if not os.path.exists(dir_gen):
  os.makedirs(dir_gen)

# Get a list of all file names in the folder
file_names = os.listdir(images_path)

# Size of the test set (note: we will perform the train/val/test split when fine-tuning Stable Diffusion so that each model is evaluated on the same test set)
num_test = 3040

# Generate a random shuffle of all data
permutations = np.random.permutation(len(file_names))

# Generate an image using the model for each prompt
for i in range(num_test):
  index = permutations[i]
  image = Image.open(images_path + file_names[index])
  image.save(dir_orig + file_names[index])
  image_id = file_names[index][0:22]
  prompt = prompts[image_id]
  generated_image = pipe(prompt).images[0]
  generated_image.save(dir_gen + file_names[index][:-4] + "generated.jpg")

# Caluclate the FID score
score = fid.compute_fid(dir_orig, dir_gen)
print("FID score: " + str(score))